In [ ]:
# 기준size (512,512,1)

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from selenium.webdriver.common.keys import Keys
from tensorflow.keras import backend as K
from keras import objectives

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,Activation,LeakyReLU,UpSampling2D,Input,Dense,Reshape,Flatten,Conv2DTranspose,ReLU,concatenate,ZeroPadding2D
import numpy as np

initializer = tf.random_normal_initializer(0.,0.02)

inputs = Input(shape=(256,256,3))
layer1 = Conv2D(filters=64,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(inputs)
layer1 = LeakyReLU()(layer1)
layer1_ = layer1

layer2 = Conv2D(filters=128,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer1)
layer2_ = BatchNormalization()(layer2)
layer2 = LeakyReLU()(layer2_)

layer3 = Conv2D(filters=256,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer2)
layer3_ = BatchNormalization()(layer3)
layer3 = LeakyReLU()(layer3_)

layer4 = Conv2D(filters=512,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer3)
layer4_ = BatchNormalization()(layer4)
layer4 = LeakyReLU()(layer4_)

layer5 = Conv2D(filters=512,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer4)
layer5_ = BatchNormalization()(layer5)
layer5 = LeakyReLU()(layer5_)

layer6 = Conv2D(filters=512,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer5)
layer6_ = BatchNormalization()(layer6)
layer6 = LeakyReLU()(layer6_)

layer7 = Conv2D(filters=512,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer6)
layer7_ = BatchNormalization()(layer7)
layer7 = LeakyReLU()(layer7_)



layer8 = Conv2D(filters=512,kernel_size=4,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer7)
layer8_ = BatchNormalization()(layer8)
layer8 = LeakyReLU()(layer8_)



layer9 = Conv2DTranspose(filters=512,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = layer9+layer7_
layer9 = Dropout(0.5)(layer9)
layer9 = ReLU()(layer9)

layer10 = Conv2DTranspose(filters=512,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer9)
layer10 = BatchNormalization()(layer10)
layer10 = concatenate([layer10,layer6_])
layer10 = Dropout(0.5)(layer10)
layer10 = ReLU()(layer10)

layer11 = Conv2DTranspose(filters=512,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = layer11+layer5_
layer11 = Dropout(0.5)(layer11)
layer11 = ReLU()(layer11)
                      
layer12 = Conv2DTranspose(filters=512,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = layer12+layer4_
layer12 = ReLU()(layer12)
                      
layer13 = Conv2DTranspose(filters=256,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = layer13+layer3_
layer13 = ReLU()(layer13)

layer14 = Conv2DTranspose(filters=128,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = layer14+layer2_
layer14 = ReLU()(layer14)

layer15 = Conv2DTranspose(filters=64,kernel_size=4,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer14)
layer15 = BatchNormalization()(layer15)
layer15 = layer15+layer1_
layer15 = ReLU()(layer15)

outputs = Conv2DTranspose(3,4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh')(layer15)

Generator = Model(inputs=inputs,outputs=outputs)
Generator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 3072        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 128, 128, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 128)  131072      leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [6]:
npy_x = ((np.load("G:/내 드라이브/P_project/"+"datasets/npy/npy_x_256_6_color.npy").reshape(-1,256,256,3)/255.0)-0.5)/0.5
npy_y = ((np.load("G:/내 드라이브/P_project/"+"datasets/npy/npy_y_256_6_color.npy")/255.0)-0.5)/0.5#[:,int(npy_x.shape[1]/4):3*int(npy_x.shape[1]/4),int(npy_x.shape[2]/4):3*int(npy_x.shape[2]/4),:]
npy_x.shape,npy_y.shape

((17857, 256, 256, 3), (17857, 256, 256, 3))

In [3]:
def pix2pix_loss(y_true, y_pred):
    y_true_flat = K.batch_flatten(y_true)
    y_pred_flat = K.batch_flatten(y_pred)

    # Adversarial Loss
    L_adv = objectives.binary_crossentropy(y_true_flat, y_pred_flat)

    # A to B loss
    # b_flat = K.batch_flatten(b)
    # bp_flat = K.batch_flatten(bp)
    L_atob = K.mean(K.abs(y_true_flat - y_pred_flat))

    return L_adv + 50 * L_atob

In [ ]:
0.022

In [7]:
def mae_standard(y_true,y_pred):
    return tf.math.reduce_mean(tf.abs(y_true-y_pred))-tf.math.reduce_std(y_pred)/8
def mse_standard(y_true,y_pred):
    return tf.math.reduce_mean(tf.square(y_true-y_pred))-tf.math.reduce_std(y_pred)/8

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_myloss_color.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)


Generator.compile(loss = pix2pix_loss,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=1000,validation_split=0.15,batch_size=16,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
Generator = DoubleBase_Unet_modeling()
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_mae_customloss_standard_8.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=80)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=17,factor=0.6,verbose=1)


Generator.compile(loss = mae_standard,optimizer = optimizer,metrics=['mae'])
Generator.fit(npy_x,npy_y,epochs=1000,validation_split=0.15,batch_size=64,callbacks=[cp,es,reLR])
Generator.summary()


'-0.5 분산오차 해바바'
'0.0076'
'0.0080'
'0.002'

Epoch 1/1000
238/238 [==============================] - 184s 703ms/step - loss: 0.1666 - mae: 0.2404 - val_loss: 0.8531 - val_mae: 0.9639
Epoch 2/1000
238/238 [==============================] - 103s 433ms/step - loss: 0.0668 - mae: 0.1454 - val_loss: 0.8186 - val_mae: 0.9353
Epoch 3/1000
238/238 [==============================] - 102s 430ms/step - loss: 0.0336 - mae: 0.1132 - val_loss: 0.8205 - val_mae: 0.9113
Epoch 4/1000
238/238 [==============================] - 103s 433ms/step - loss: 0.0229 - mae: 0.1027 - val_loss: 0.8497 - val_mae: 0.9730
Epoch 5/1000
238/238 [==============================] - 103s 432ms/step - loss: 0.0158 - mae: 0.0953 - val_loss: 0.8098 - val_mae: 0.9296
Epoch 6/1000
238/238 [==============================] - 103s 432ms/step - loss: 0.0141 - mae: 0.0938 - val_loss: 0.8207 - val_mae: 0.9441
Epoch 7/1000
238/238 [==============================] - 103s 433ms/step - loss: 0.0113 - mae: 0.0912 - val_loss: 1.0070 - val_mae: 1.0786
Epoch 8/1000
238/238 [============

238/238 [==============================] - 103s 433ms/step - loss: -0.0376 - mae: 0.0423 - val_loss: 0.8657 - val_mae: 0.8657
Epoch 60/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0369 - mae: 0.0427 - val_loss: 0.4268 - val_mae: 0.5183
Epoch 61/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0378 - mae: 0.0420 - val_loss: 0.5338 - val_mae: 0.6216
Epoch 62/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0376 - mae: 0.0422 - val_loss: 0.9273 - val_mae: 0.9874
Epoch 63/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0374 - mae: 0.0421 - val_loss: 1.0857 - val_mae: 1.1097
Epoch 64/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0382 - mae: 0.0415 - val_loss: 0.6795 - val_mae: 0.7365
Epoch 65/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0384 - mae: 0.0413 - val_loss: 0.8612 - val_mae: 0.9194

Epoch 00065: ReduceLROnPlateau re

238/238 [==============================] - 103s 433ms/step - loss: -0.0445 - mae: 0.0351 - val_loss: 0.5835 - val_mae: 0.6584
Epoch 117/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0443 - mae: 0.0354 - val_loss: 0.5855 - val_mae: 0.6531
Epoch 118/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0442 - mae: 0.0354 - val_loss: 0.6575 - val_mae: 0.7287
Epoch 119/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0445 - mae: 0.0351 - val_loss: 0.4444 - val_mae: 0.5474
Epoch 120/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0447 - mae: 0.0350 - val_loss: 0.3579 - val_mae: 0.4633
Epoch 121/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0441 - mae: 0.0355 - val_loss: 0.3591 - val_mae: 0.4585
Epoch 122/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0435 - mae: 0.0361 - val_loss: 0.7717 - val_mae: 0.8709
Epoch 123/1000
238/238 [====

238/238 [==============================] - 103s 433ms/step - loss: -0.0469 - mae: 0.0328 - val_loss: 0.1034 - val_mae: 0.1791
Epoch 173/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0469 - mae: 0.0328 - val_loss: 0.1981 - val_mae: 0.2934
Epoch 174/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0466 - mae: 0.0330 - val_loss: 0.0949 - val_mae: 0.1725
Epoch 175/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0466 - mae: 0.0329 - val_loss: 0.2572 - val_mae: 0.3249
Epoch 176/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0464 - mae: 0.0332 - val_loss: 0.1713 - val_mae: 0.2547
Epoch 177/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0471 - mae: 0.0327 - val_loss: 0.2873 - val_mae: 0.3682
Epoch 178/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0469 - mae: 0.0328 - val_loss: 0.0940 - val_mae: 0.1665
Epoch 179/1000
238/238 [====

238/238 [==============================] - 103s 431ms/step - loss: -0.0473 - mae: 0.0322 - val_loss: 0.0605 - val_mae: 0.1343
Epoch 230/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0474 - mae: 0.0322 - val_loss: 0.0695 - val_mae: 0.1519
Epoch 231/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0476 - mae: 0.0321 - val_loss: 0.0668 - val_mae: 0.1429
Epoch 232/1000
238/238 [==============================] - 103s 432ms/step - loss: -0.0482 - mae: 0.0319 - val_loss: 0.0255 - val_mae: 0.1027
Epoch 233/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0478 - mae: 0.0320 - val_loss: 0.1185 - val_mae: 0.1897

Epoch 00233: ReduceLROnPlateau reducing learning rate to 3.023308381671086e-05.
Epoch 234/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.0475 - mae: 0.0322 - val_loss: 0.0808 - val_mae: 0.1662
Epoch 235/1000
238/238 [==============================] - 103s 433ms/step - loss: -0.047

 42/238 [====>.........................] - ETA: 1:19 - loss: -0.0483 - mae: 0.0316

In [7]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
Generator = DoubleBase_Unet_modeling()
optimizer = Adam(lr=0.002)
modelpath = "./datasets/models/pix2pix_mse_customloss_standard_20.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=150)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=21,factor=0.8,verbose=1)


Generator.compile(loss = mse_standard,optimizer = optimizer,metrics=['mse'])
Generator.fit(npy_x,npy_y,epochs=3000,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])
Generator.summary()

'mse로 했었어 그래서 mse+분산으로할거야'
'0.0165'

Epoch 1/3000
475/475 [==============================] - 138s 283ms/step - loss: 0.1314 - mse: 0.1561 - val_loss: 1.7367 - val_mse: 1.7368
Epoch 2/3000
475/475 [==============================] - 124s 261ms/step - loss: 0.0123 - mse: 0.0434 - val_loss: 1.7367 - val_mse: 1.7368
Epoch 3/3000
475/475 [==============================] - 124s 260ms/step - loss: -0.0050 - mse: 0.0267 - val_loss: 1.7368 - val_mse: 1.7368
Epoch 4/3000
475/475 [==============================] - 124s 262ms/step - loss: -0.0105 - mse: 0.0213 - val_loss: 1.7367 - val_mse: 1.7368
Epoch 5/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0153 - mse: 0.0167 - val_loss: 1.7368 - val_mse: 1.7368
Epoch 6/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0163 - mse: 0.0158 - val_loss: 1.7368 - val_mse: 1.7368
Epoch 7/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0183 - mse: 0.0139 - val_loss: 1.7368 - val_mse: 1.7368
Epoch 8/3000
475/475 [=======

475/475 [==============================] - 124s 261ms/step - loss: -0.0290 - mse: 0.0036 - val_loss: 0.5615 - val_mse: 0.6073
Epoch 116/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0289 - mse: 0.0036 - val_loss: 0.2644 - val_mse: 0.3017
Epoch 117/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0290 - mse: 0.0036 - val_loss: 0.2854 - val_mse: 0.3184
Epoch 118/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0291 - mse: 0.0035 - val_loss: 0.2751 - val_mse: 0.3121
Epoch 119/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0291 - mse: 0.0035 - val_loss: 1.4505 - val_mse: 1.4751
Epoch 120/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0288 - mse: 0.0037 - val_loss: 0.0952 - val_mse: 0.1279
Epoch 121/3000
475/475 [==============================] - 124s 261ms/step - loss: -0.0289 - mse: 0.0035 - val_loss: 0.6505 - val_mse: 0.6765
Epoch 122/3000
475/475 [====

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_mae_color.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)


Generator.compile(loss = 'mae',optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=1000,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_mse_mae_color.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)


Generator.compile(loss = mae_and_mse,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=100,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
def mae_and_mse(y_true,y_pred):
    return tf.math.reduce_mean(tf.square(y_true-y_pred)) + *tf.math.reduce_mean(tf.abs(y_true-y_pred))

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_mse_2mae.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)


Generator.compile(loss = mae_and_mse,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=100,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
def mae_and_mse5(y_true,y_pred):
    return tf.math.reduce_mean(tf.square(y_true-y_pred)) + 25*tf.math.reduce_mean(tf.abs(y_true-y_pred))

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "./datasets/models/pix2pix_mse_25mae.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)


Generator.compile(loss = mae_and_mse5,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=100,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
Generator = DoubleBase_Unet_modeling()
optimizer = Adam(lr=0.003)
modelpath = "C:/models/double_model.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)

Generator = DoubleBase_Unet_modeling()
Generator.compile(loss = pix2pix_loss,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=1000,validation_split=0.15,batch_size=16,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten,AveragePooling2D,Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.003)
modelpath = "C:/models/double_model_double.h5"
cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
es = EarlyStopping(monitor = "val_loss",patience=50)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=13,factor=0.6)

Generator = ReverseUnet_modeling()
Generator.compile(loss = pix2pix_loss,optimizer = optimizer)
Generator.fit(npy_x,npy_y,epochs=1000,validation_split=0.15,batch_size=16,callbacks=[cp,es,reLR])
Generator.summary()


'868'

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,Activation,LeakyReLU,UpSampling2D,Input,Dense,Reshape,Flatten,Conv2DTranspose,ReLU,concatenate,ZeroPadding2D
import numpy as np

def DoubleBase_Unet_modeling(kernel_size=4,drop_out=0.5):
    initializer = tf.random_normal_initializer(0.,0.02)
    inputs = Input(shape=(256,256,3))
    layer1 = Conv2D(filters=128,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(inputs)
    layer1 = LeakyReLU()(layer1)
    layer1_ = layer1

    layer2 = Conv2D(filters=256,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer1)
    layer2_ = BatchNormalization()(layer2)
    layer2 = LeakyReLU()(layer2_)

    layer3 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer2)
    layer3_ = BatchNormalization()(layer3)
    layer3 = LeakyReLU()(layer3_)

    layer4 = Conv2D(filters=1024,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer3)
    layer4_ = BatchNormalization()(layer4)
    layer4 = LeakyReLU()(layer4_)

    layer5 = Conv2D(filters=1024,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer4)
    layer5_ = BatchNormalization()(layer5)
    layer5 = LeakyReLU()(layer5_)

    layer6 = Conv2D(filters=1024,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer5)
    layer6_ = BatchNormalization()(layer6)
    layer6 = LeakyReLU()(layer6_)

    layer7 = Conv2D(filters=1024,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer6)
    layer7_ = BatchNormalization()(layer7)
    layer7 = LeakyReLU()(layer7_)



    layer8 = Conv2D(filters=1024,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer7)
    layer8_ = BatchNormalization()(layer8)
    layer8 = LeakyReLU()(layer8_)



    layer9 = Conv2DTranspose(filters=1024,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer8)
    layer9 = BatchNormalization()(layer9)
    layer9 = layer9+layer7_
    layer9 = Dropout(drop_out)(layer9)
    layer9 = ReLU()(layer9)

    layer10 = Conv2DTranspose(filters=1024,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer9)
    layer10 = BatchNormalization()(layer10)
    layer10 = concatenate([layer10,layer6_])
    layer10 = Dropout(drop_out)(layer10)
    layer10 = ReLU()(layer10)

    layer11 = Conv2DTranspose(filters=1024,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer10)
    layer11 = BatchNormalization()(layer11)
    layer11 = layer11+layer5_
    layer11 = Dropout(drop_out)(layer11)
    layer11 = ReLU()(layer11)

    layer12 = Conv2DTranspose(filters=1024,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer11)
    layer12 = BatchNormalization()(layer12)
    layer12 = layer12+layer4_
    layer12 = ReLU()(layer12)

    layer13 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer12)
    layer13 = BatchNormalization()(layer13)
    layer13 = layer13+layer3_
    layer13 = ReLU()(layer13)

    layer14 = Conv2DTranspose(filters=256,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer13)
    layer14 = BatchNormalization()(layer14)
    layer14 = layer14+layer2_
    layer14 = ReLU()(layer14)

    layer15 = Conv2DTranspose(filters=128,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer14)
    layer15 = BatchNormalization()(layer15)
    layer15 = layer15+layer1_
    layer15 = ReLU()(layer15)

    outputs = Conv2DTranspose(3,4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh')(layer15)

    Generator = Model(inputs=inputs,outputs=outputs)
    return Generator

#model = DoubleBase_Unet_modeling()
#model.summary()

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,Activation,LeakyReLU,UpSampling2D,Input,Dense,Reshape,Flatten,Conv2DTranspose,ReLU,concatenate,ZeroPadding2D
import numpy as np

def Reverse_Unet_down_modeling(kernel_size=4,drop_out=0.5):
    initializer = tf.random_normal_initializer(0.,0.02)
    inputs = Input(shape=(256,256,3))
    layer1 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(inputs)
    layer1 = LeakyReLU()(layer1)
    layer1_ = layer1

    layer2 = Conv2DTranspose(filters=256,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer1)
    layer2_ = BatchNormalization()(layer2)
    layer2 = LeakyReLU()(layer2_)

    layer3 = Conv2DTranspose(filters=128,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer2)
    layer3_ = BatchNormalization()(layer3)
    layer3 = LeakyReLU()(layer3_)

    layer4 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer3)
    layer4_ = BatchNormalization()(layer4)
    layer4 = LeakyReLU()(layer4_)

    layer5 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer4)
    layer5_ = BatchNormalization()(layer5)
    layer5 = LeakyReLU()(layer5_)

    layer6 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer5)
    layer6_ = BatchNormalization()(layer6)
    layer6 = LeakyReLU()(layer6_)

    layer7 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer6)
    layer7_ = BatchNormalization()(layer7)
    layer7 = LeakyReLU()(layer7_)



    layer8 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer7)
    layer8_ = BatchNormalization()(layer8)
    layer8 = LeakyReLU()(layer8_)



    layer9 = Conv2D(filters=64,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer8)
    layer9 = BatchNormalization()(layer9)
    layer9 = layer9+layer7_
    layer9 = Dropout(drop_out)(layer9)
    layer9 = ReLU()(layer9)

    layer10 = Conv2D(filters=64,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer9)
    layer10 = BatchNormalization()(layer10)
    layer10 = concatenate([layer10,layer6_])
    layer10 = Dropout(drop_out)(layer10)
    layer10 = ReLU()(layer10)

    layer11 = Conv2D(filters=64,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer10)
    layer11 = BatchNormalization()(layer11)
    layer11 = layer11+layer5_
    layer11 = Dropout(drop_out)(layer11)
    layer11 = ReLU()(layer11)

    layer12 = Conv2D(filters=64,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer11)
    layer12 = BatchNormalization()(layer12)
    layer12 = layer12+layer4_
    layer12 = ReLU()(layer12)

    layer13 = Conv2D(filters=128,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer12)
    layer13 = BatchNormalization()(layer13)
    layer13 = layer13+layer3_
    layer13 = ReLU()(layer13)

    layer14 = Conv2D(filters=256,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer13)
    layer14 = BatchNormalization()(layer14)
    layer14 = layer14+layer2_
    layer14 = ReLU()(layer14)

    layer15 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer14)
    layer15 = BatchNormalization()(layer15)
    layer15 = layer15+layer1_
    layer15 = ReLU()(layer15)

    outputs = Conv2D(3,4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh')(layer15)

    Generator = Model(inputs=inputs,outputs=outputs)
    return Generator

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,Activation,LeakyReLU,UpSampling2D,Input,Dense,Reshape,Flatten,Conv2DTranspose,ReLU,concatenate,ZeroPadding2D
import numpy as np

def ReverseUnet_modeling(kernel_size=4,drop_out=0.5):
    initializer = tf.random_normal_initializer(0.,0.02)
    inputs = Input(shape=(256,256,3))
    layer1 = Conv2DTranspose(filters=64,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(inputs)
    layer1 = LeakyReLU()(layer1)
    layer1_ = layer1

    layer2 = Conv2DTranspose(filters=128,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer1)
    layer2_ = BatchNormalization()(layer2)
    layer2 = LeakyReLU()(layer2_)

    layer3 = Conv2DTranspose(filters=256,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer2)
    layer3_ = BatchNormalization()(layer3)
    layer3 = LeakyReLU()(layer3_)

    layer4 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer3)
    layer4_ = BatchNormalization()(layer4)
    layer4 = LeakyReLU()(layer4_)

    layer5 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer4)
    layer5_ = BatchNormalization()(layer5)
    layer5 = LeakyReLU()(layer5_)

    layer6 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer5)
    layer6_ = BatchNormalization()(layer6)
    layer6 = LeakyReLU()(layer6_)

    layer7 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer6)
    layer7_ = BatchNormalization()(layer7)
    layer7 = LeakyReLU()(layer7_)



    layer8 = Conv2DTranspose(filters=512,kernel_size=kernel_size,strides=2,padding='same',use_bias=False,kernel_initializer=initializer)(layer7)
    layer8_ = BatchNormalization()(layer8)
    layer8 = LeakyReLU()(layer8_)



    layer9 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer8)
    layer9 = BatchNormalization()(layer9)
    layer9 = layer9+layer7_
    layer9 = Dropout(drop_out)(layer9)
    layer9 = ReLU()(layer9)

    layer10 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer9)
    layer10 = BatchNormalization()(layer10)
    layer10 = concatenate([layer10,layer6_])
    layer10 = Dropout(drop_out)(layer10)
    layer10 = ReLU()(layer10)

    layer11 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer10)
    layer11 = BatchNormalization()(layer11)
    layer11 = layer11+layer5_
    layer11 = Dropout(drop_out)(layer11)
    layer11 = ReLU()(layer11)

    layer12 = Conv2D(filters=512,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer11)
    layer12 = BatchNormalization()(layer12)
    layer12 = layer12+layer4_
    layer12 = ReLU()(layer12)

    layer13 = Conv2D(filters=256,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer12)
    layer13 = BatchNormalization()(layer13)
    layer13 = layer13+layer3_
    layer13 = ReLU()(layer13)

    layer14 = Conv2D(filters=128,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer13)
    layer14 = BatchNormalization()(layer14)
    layer14 = layer14+layer2_
    layer14 = ReLU()(layer14)

    layer15 = Conv2D(filters=64,kernel_size=kernel_size,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(layer14)
    layer15 = BatchNormalization()(layer15)
    layer15 = layer15+layer1_
    layer15 = ReLU()(layer15)

    outputs = Conv2D(3,4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh')(layer15)

    Generator = Model(inputs=inputs,outputs=outputs)
    return Generator